In [2]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import json
import html



yillar = list(range(2023, 2018, -1))
tum_maclar = []

##Buraya istediğin bir ligi koyabilirsin.Tek yapman gereken FBref sitesinden ilgili ligin 
#puan tablosunun bulunduğu linki kopyalamak.GOl beklentisi geçmiş yıllar için olmayabilir, buna dikkat et!
hedef_url = "https://fbref.com/en/comps/22/Major-League-Soccer-Stats"
import time
for yil in yillar:
    data = requests.get(hedef_url)
    soup = BeautifulSoup(data.text)
    tablo = soup.select('table.stats_table')[0]
    linkler = [l.get("href") for l in tablo.find_all('a')]
    linkler = [l for l in linkler if '/squads/' in l]
    takim_urlleri = [f"https://fbref.com{l}" for l in linkler]
    onceki_sezon = soup.select("a.prev")[0].get("href")
    hedef_url = f"https://fbref.com{onceki_sezon}"
    for takim_url in takim_urlleri:
        team_name = takim_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(takim_url)
        maclar = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        linkler = [l.get("href") for l in soup.find_all('a')]
        linkler = [l for l in linkler if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{linkler[0]}")
        sut= pd.read_html(data.text, match="Shooting")[0]
        sut.columns = sut.columns.droplevel()
    
        try:
            takim_verisi = maclar.merge(sut[["Date","SoT%","Dist"]], on="Date")
        except ValueError:
            continue
        
        # takim_verisi =takim_verisi[takim_verisi["Comp"] == "Premier League"]
        takim_verisi["Season"] = yil
        takim_verisi["Team"] = team_name
        tum_maclar.append(takim_verisi)
    
        time.sleep(5)

In [ ]:

match_df = pd.concat(tum_maclar)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("match_data.csv")

In [12]:

# # match_df = pd.concat(tum_maclar)
# # match_df.columns = [c.lower() for c in match_df.columns]
# # match_df.to_csv("match_data.csv")